# 플라스크 임포트  

In [1]:
import numpy as np

In [2]:
val = np.float64(0)
pyval = val.item()

In [3]:
!pip install PyMySQL

In [4]:
from flask import Flask
from flask import redirect, request , jsonify

from flask_cors import CORS
import pandas as pd
import pickle
from konlpy.tag import Okt, Kkma
okt = Okt()

from sklearn.feature_extraction.text import CountVectorizer

# PymySQL

In [5]:
import pymysql

In [6]:
db = pymysql.connect(host = 'project-db-stu.ddns.net', port = 3307, user= 'dodam',
                    passwd='115', db='dodam', charset='utf8')
db

In [7]:
cursor = db.cursor(pymysql.cursors.DictCursor)
cursor

In [8]:
sql = "SELECT * FROM `t_diary`;"
cursor.execute(sql)
result = cursor.fetchall()
result

[{'d_seq': 1,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 !!',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 15)},
 {'d_seq': 2,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 !!',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 16)},
 {'d_seq': 3,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 !!',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 17)},
 {'d_seq': 4,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 !!',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 18)},
 {'d_seq': 5,
  'm_id': 'admin',
  'c_seq': 1,
  'd_title': '랄랄라 즐거운 스마트 인재 개발원 !!',
  'd_content': '오늘은 점심은 또 뭐먹을까 ',
  'd_msg': '1515 볶음밥은 맛있어 ',
  'd_date': datetime.date(2022, 4, 19)}]

In [9]:
def myTokenizer2(text):
    d = pd.DataFrame(okt.pos(text, norm=True, stem=True), columns=['morph','tag'])
    d.set_index('tag', inplace=True)
    if ('Noun' in d.index) | ('Verb' in d.index) | ('Adjective' in d.index):
        labels = ['Noun','Verb','Adjective']
        return d.loc[d.index.intersection(labels)].dropna()['morph'].values
    else :
        return []

In [10]:
def height_predict(model, input_height, n=72):
    max_len = 72
    for _ in range(len(input_height),n):# n번 반복
        encoded = pad_sequences(input_height, maxlen=max_len-1, padding='pre',dtype='float32').reshape(71,-1)
        result = model.predict([encoded.tolist()],verbose=0)
        input_height = np.hstack([input_height,result])
    return input_height

In [11]:
# 수정
result = {}
app = Flask(__name__)
CORS(app)

# with open('dodam_model.pkl', 'rb') as f:
#     model = pickle.load(f)
    
@app.route("/post2",  methods=["POST", "GET"])
def test():
    if (request.method=="POST"):
#        display(request.form)
        #result = preprocessing(request.form.get('diary'))
        display(request.form)
        m_id=request.form.get('m_id')
        c_seq=request.form.get('c_seq')
        d_title=request.form.get('d_title')
        diary=request.form.get('diary')
        d_msg=request.form.get('d_msg')
        d_date=request.form.get('d_date')
        sql = '''insert into `t_diary` (m_id, c_seq, d_title, d_content, d_msg, d_date) 
values(%s, %s, %s, %s, %s, %s)'''
        cursor.execute(sql , (m_id, c_seq, d_title, diary, d_msg, d_date))
        db.commit()
        result = cursor.fetchall()
        result
        #텍스트마이닝 
        result = request.form.get('d_content')
#         with open('dodam_cv.pkl', 'rb') as f:
#     # 정의된 모델 선언
#             cv = pickle.load(f)
#     # 사용자가 입력한 데이터를 모델이 읽을 수 있는 데이터로 변환
#             data_review = cv.transform([result])
#     # 예측 결과를 a에 담는다.
        with open('dodam_lr.pkl','rb') as f1:
              lr_model = pickle.load(f1)
                pre = lr_model.predict_proba(data_review)
                feel_list = ['기쁨','분노','불안','슬픔']
                result_list = []
                for i in range(len(feel_list)):
                    result_list.append(pre[0][i])
                    print(f"{feel_list[i]} : {pre[0][i]}")
                e_joy = round((result_list[0]*100),2)
                e_joy = e_joy.item()
                e_sorrow = round((result_list[3]*100),2)
                e_sorrow = e_sorrow.item()
                print(e_sorrow)
                print(type(e_sorrow))
                e_anger = round((result_list[1]*100),2)
                e_anger = e_anger.item()
                e_unrest = round((result_list[2]*100),2)
                e_unrest = e_unrest.item()
                sql1 = '''insert into `t_emotion` (m_id, e_joy, e_sorrow, e_anger, e_unrest, d_date) 
values(%s, %s, %s, %s, %s, %s)'''
                cursor.execute(sql1, (m_id, e_joy, e_sorrow, e_anger, e_unrest, d_date))
                db.commit()
                result1 = cursor.fetchall()
                result1
                print(pre[0])
#         result = test()
#         print(result)
        result2 = str("%.2f%%"%(pre[0][0]*100)) + "<br>" + str(feel_list[1])+" : "+str("%.2f%%"%(pre[0][1]*100)) + "<br>"+str(feel_list[2])+" : "+str("%.2f%%"%(pre[0][2]*100)) + "<br>"+str(feel_list[3])+" : "+str("%.2f%%"%(pre[0][3]*100))
        #return str(feel_list[0])+" : "
        return redirect("http://localhost:8089/dodam/diary.do?result2="+result2)
    else:
        return "왜안돼?"

@app.route("/height",  methods=["POST", "GET"])
def test2():
    if (request.method=="POST"):
#        display(request.form)
        #result = preprocessing(request.form.get('diary'))
        model = load_model('Height.h5')   
        display(request.form)
        input_height = request.form.get('height')
        print(input_height)
        result = height_predict(model, np.array([[input_height]]))
        print(result)
        return str(float(result[0][71])*100//1/100)
    else:
        return "왜안돼?"

if __name__ == "__main__":
    app.run(host = "127.0.0.1", port =7000)
    # result = test()

IndentationError: unexpected indent (Temp/ipykernel_2088/1914634602.py, line 35)